In [64]:
import pandas as pd
from bokeh.io import output_file, show ,curdoc
from bokeh.plotting import figure
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper, Slider, Select
from bokeh.palettes import Spectral6
from bokeh.layouts import widgetbox , row , column

In [65]:
data = pd.read_csv('gapminder_tidy.csv' )
data.set_index("Year", inplace = True)
regions_list = data.region.unique().tolist()
data.head()

,Country,fertility,life,population,child_mortality,gdp,region
Year,,,,,,,
1964,Afghanistan,7.671,33.639,10474903.0,339.7,1182.0,South Asia
1965,Afghanistan,7.671,34.152,10697983.0,334.1,1182.0,South Asia
1966,Afghanistan,7.671,34.662,10927724.0,328.7,1168.0,South Asia
1967,Afghanistan,7.671,35.170,11163656.0,323.3,1173.0,South Asia
1968,Afghanistan,7.671,35.674,11411022.0,318.1,1187.0,South Asia


In [66]:
#callback function
def update_plot(attr, old, new):
    yr = slider.value
    x = x_select.value
    y = y_select.value
   
    plot.xaxis.axis_label = x
    plot.yaxis.axis_label = y
   
    new_data = {
        'x'       : data.loc[yr][x],
        'y'       : data.loc[yr][y],
        'country' : data.loc[yr].Country,
        'pop'     : (data.loc[yr].population / 20000000) + 2,
        'region'  : data.loc[yr].region,
    }
   
    source.data = new_data

    
    plot.x_range.start = min(data[x])
    plot.x_range.end = max(data[x])
    plot.y_range.start = min(data[y])
    plot.y_range.end = max(data[y])

    
    plot.title.text = 'Gapminder data for %d' % yr


In [67]:
#initial value of source
source = ColumnDataSource(data={
    'x'       : data.loc[1970].fertility,
    'y'       : data.loc[1970].life,
    'country' : data.loc[1970].Country,
    'pop'     : (data.loc[1970].population / 20000000) + 2,
    'region'  : data.loc[1970].region,
})

In [68]:
color_mapper = CategoricalColorMapper(factors=regions_list, palette=Spectral6)

In [69]:
#select box
x_select = Select(
    options=['fertility', 'life', 'child_mortality', 'gdp'],
    value='fertility',
    title='x-axis data'
)
x_select.on_change('value', update_plot)
y_select = Select(
    options=['fertility', 'life', 'child_mortality', 'gdp'],
    value='life',
    title='y-axis data'
)
y_select.on_change('value', update_plot)


In [70]:
xmin, xmax = min(data.fertility), max(data.fertility)
ymin, ymax = min(data.life), max(data.life)

In [71]:
plot = figure(title='Gapminder Data for 1970', plot_height=400, plot_width=700,
              x_range=(xmin, xmax), y_range=(ymin, ymax) )


In [72]:
plot.circle(x='x', y='y', fill_alpha=0.8, source=source,
            color=dict(field='region', transform=color_mapper), legend='region')


GlyphRenderer(id='1748', ...)

In [73]:
#creating a slider and hover
slider = Slider(start=1970 , end =2010 , step=1 , value=1970 , title='Year')
slider.on_change('value', update_plot)
hover = HoverTool(tooltips=[('Country','@country')])
plot.add_tools(hover)

In [74]:
plot.legend.location = 'top_right'
plot.xaxis.axis_label ='Fertility (children per woman)'
plot.yaxis.axis_label = 'Life Expectancy (years) '

In [75]:
output_file("gapminder.html")
layout = row(widgetbox(slider, x_select, y_select), plot)
curdoc().add_root(layout)

